In [1]:
import pandas as pd
import numpy as np

import pickle

In [3]:
preds_path_ls = [
    ('raw', r'C:\bogachevv\Python\OrthogonalFineTune\test_preds\raw.bin'),
    ('LoRA', r'C:\bogachevv\Python\OrthogonalFineTune\test_preds\LoRA.bin')
]

In [4]:
preds_dfs = {}

for run_name, preds_path in preds_path_ls:
    with open(preds_path, 'rb') as f:
        preds_dfs[run_name] = pickle.load(file=f)

In [6]:
preds_dfs['LoRA'].head()

,generated_text,subject,pred,true,corr,category
0,B. 4,abstract_algebra,B,B,1,STEM
1,C. 24,abstract_algebra,C,C,1,STEM
2,C. 0,abstract_algebra,C,D,0,STEM
3,"A. True,",abstract_algebra,A,B,0,STEM
4,B. 6,abstract_algebra,B,B,1,STEM


In [7]:
acc_by_subjects = pd.DataFrame({
    'subject': list(set(preds_dfs['raw'].subject))
})

for run_name, preds_df in preds_dfs.items():
    acc_df = preds_df[['subject', 'corr']].groupby(['subject'], as_index=False).mean()
    acc_df.rename(columns={'corr': run_name}, inplace=True)

    acc_by_subjects = acc_by_subjects.merge(
        acc_df,
        left_on='subject',
        right_on='subject',
        how='right'
    )

In [8]:
acc_by_subjects

,subject,raw,LoRA
0,abstract_algebra,0.350000,0.400000
1,anatomy,0.674074,0.622222
2,astronomy,0.750000,0.710526
3,business_ethics,0.660000,0.620000
4,clinical_knowledge,0.716981,0.735849
5,college_biology,0.715278,0.791667
6,college_chemistry,0.450000,0.460000
7,college_computer_science,0.470000,0.480000
8,college_mathematics,0.350000,0.330000
9,college_medicine,0.618497,0.624277


In [9]:
acc_by_categories = pd.DataFrame({
    'category': list(set(preds_dfs['raw'].category))
})

for run_name, preds_df in preds_dfs.items():
    acc_df = preds_df[['category', 'corr']].groupby(['category'], as_index=False).mean()
    acc_df.rename(columns={'corr': run_name}, inplace=True)

    acc_by_categories = acc_by_categories.merge(
        acc_df,
        left_on='category',
        right_on='category',
        how='right'
    )

In [10]:
acc_by_categories

,category,raw,LoRA
0,STEM,0.525182,0.542744
1,humanities,0.527099,0.568757
2,"other (business, health, misc.)",0.706663,0.705737
3,social sciences,0.721807,0.746831


In [9]:
total_res = pd.DataFrame(columns=['run_name', 'accuracy', 'correctness'])

for run_name, preds_df in preds_dfs.items():
    total_accuracy = preds_df['corr'].mean()
    total_correctness = (preds_df['pred'] != 'I').mean()

    total_res = pd.concat([
        total_res,
        pd.DataFrame({
            'run_name': [run_name],
            'accuracy': [total_accuracy],
            'correctness': [total_correctness],
        })
    ])

C:\Users\Vladimir\AppData\Local\Temp\ipykernel_24520\2066953499.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_res = pd.concat([


In [10]:
total_res

,run_name,accuracy,correctness
0,raw,0.606793,0.996734
0,cfg11,0.606793,0.990202
0,cfg12,0.610059,0.984977
0,cfg13,0.636839,0.992815
0,test_cfg13,0.633813,0.995656
